# This notebook is intended to present how to produce a point-wise differential relative entropy analysis over intensity maps generated through $\texttt{limlam_mocker}$

#### Author: Patrick Horlaville

First, read this notebook from the root limCode folder, so the imports work correctly. Therefore, this whole notebook will run as though its location is:

In [4]:
%cd '/fs/lustre/scratch/horlaville/clara_limlam/limCode2020-master_clara_2/'

/fs/lustre/scratch/horlaville/clara_limlam/limCode2020-master_clara_2


Then, import all packages:

In [6]:
# base imports
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# various math package imports
from scipy import special
from scipy import interpolate
import astropy.units as u 
from scipy.ndimage import gaussian_filter

# base intensity mapping package
from lim import lim 

# plotting preferences
matplotlib.rcParams.update({'font.size': 18,'figure.figsize':[8,7]}) 
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# used to check on the running time of particular computations
from datetime import datetime

Then, we can load the lim model. The argument of $\texttt{lim()}$ is the model chosen in $\texttt{params.py}$. Each model is designed to generate intensity maps of a particular line, more particularly a cube of intensites whose voxels are distributed along each angular coordinate (right ascension (RA) and declination) as well as in frequency space, although $\texttt{limlam_mocker}$ allows to generate other by-products, such as the one-dimensional power spectrum of the cube's intensities. In the current $\texttt{params}$ file, there are lots of models available; I personally mainly worked with mine (which I iterately worked over, the final version of which is $\texttt{Lichen_v4}$, which models the $[\rm{C_{II}}]$ line) and the Tony Li' $\rm{CO}$ model ($\texttt{TonyLi_PhI}$).

In [7]:
# Initialize LineModel with default model
m = lim()
m_cii = lim('Lichen_v4', doSim = True)

Input cosmological model does not match simulations
Setting analytic cosmology to match simulation


## Parameters' breakdown:

$\texttt{m_cii}$ is the $\texttt{lim}$ object out of which we will extract our intensity map. There are multiple input parameters which will define the cube of intensities:

1) $\texttt{catalogue_file}$: string, path to input halo lightcone 
- The most important parameter is the halo lightcone used for the simulation. The halo lightcone must contain all the information required to model the signal from all its halos (position, comoving coordinates, redshift, mass, etc.). For more information on the lightcone properties needed to be processed through $\texttt{limlam_mocker}$, refer to '/mnt/scratch-lustre/horlaville/nate_sims/gaussian/pksc3npz.py', which is the example of a script that converts raw peakpatch lightcones to npz files that can be processed by $\texttt{limlam_mocker}$.

2) $\texttt{model_name}$: string, label of the mass-luminosity function
- The second most important parameter is the mass-luminosity function used for the simulation. It determines the intensity map's line model (such as $\texttt{'Lichen_v4'}$ for my $\rm{[C_{II}]}$ model or $\texttt{'TonyLi_PhI'}$ for $\rm{CO}$). Note than in the code above, I loaded my lim object with the argument $\texttt{'Lichen_v4'}$, so the model name is already chosen.

3) $\texttt{model_par}$: dictionary, parameters for $\texttt{model_name}$
- There are typically plenty of parameters prescribed inside the mass-luminosity function, some of which can be assigned in $\texttt{model_par}$. Typically, for relative entropy analysis, those are the fiducial values to be modified. The idea is to modify them a bit and analyze how much their variation affects the simulation results. 

Cube parameters in frequency (perpendicular) space:

4) $\texttt{nu}$: astropy quantity, restframe frequency of the line
- $\texttt{nu}$ has to be set to the restframe frequency of the line prescribed from the chosen mass-luminosity function. In my case, since my mass-luminosity function targets $[\rm{C_{II}}]$, $\texttt{nu}$ is set to $\nu_{[\rm{C_{II}}],\ rest}$, which is 1897GHz.

5) $\texttt{nuObs}$: astropy quantity, median observed frequency for the cube of intensities
- $\texttt{nuObs}$ is the central observed frequency for the simulation. If the lightcone's halos span redshift [$z_0$, $z_f$], such that the median redshift is $z_{\rm{med}} = (z_0 + z_f)/2$, then $\texttt{nuObs}$ would typically be set to $\texttt{nu}/(1 + z_{\rm{med}})$.

6) $\texttt{Delta_nu}$: astropy quantity, frequency span of the cube
- The halo lightcone spanning [$z_0$, $z_f$] in redshift space, what's the span in frequency space? That would be $\nu_{\rm{max}} - \nu_{\rm{min}}$, where the maximum frequency is attained at the minimum redshift: $\nu_{\rm{max}} = \texttt{nu}/(1+z_0)$ and vice-versa: $\nu_{\rm{min}} = \texttt{nu}/(1+z_f)$. If we want our cube of intensities to span the frequency range of our halo lightcone, which is typically the case, we just set $\texttt{Delta_nu}$ to $\nu_{\rm{max}} - \nu_{\rm{min}}$. 

7) $\texttt{dnu}$: astropy quantity, frequency channelization
- Each frequency slice of the cube of intensities will be separated by $\texttt{dnu}$. Its value is usually chosen in accordance to the frequency resolution abilities of typical observational instruments aiming the observation of the desired line. For example, for $\rm{[C_{II}]}$, I ran all of my analysis using the CCAT deep spectroscopic survey osbervational characteristics, whose expected frequency channelization for $\rm{[C_{II}]}$ should be around 2.8GHz.

NOTE:

- For sharper band analysis, $\texttt{Delta_nu}$ can be smaller than $\nu_{\rm{max}} - \nu_{\rm{min}}$. Also, $\texttt{nuObs}$ can be set to any value between $\nu_{\rm{min}}$ and $\nu_{\rm{min}}$, but you have to make sure that $\texttt{Delta_nu}$ is chosen so that $\texttt{nuObs}$ + $\texttt{Delta_nu}$/2 < $\nu_{\rm{max}}$ and $\texttt{nuObs}$ - $\texttt{Delta_nu}$/2 > $\nu_{\rm{min}}$. In other words, it's important to generate a cube of intensities whose frequency span is within the frequency span of the halo lightcone. Otherwise, some of the cube's frequency channels will be outside of the frequency range of the lightcone, and there will be no halos there (although the code will still run).

Cube parameters in angular (parallel) space:

8) $\texttt{Omega_field}$: astropy quantity, angular area coverage
- $\texttt{Omega_field}$ is the angular area coverage, in the sky, of the cube. My typical process to find it from scratch is to use the halo lightcone's comoving radial distance and perpendicular size, say $\chi_r$ and $\chi_\perp$. I approximate the angular size of the halo lightcone with $\rm{tan}(\theta) \sim \frac{\chi_\perp}{\chi_r}$, such that $\theta \sim \rm{arctan}(\frac{\chi_\perp}{\chi_r})$. Then, set $\texttt{Omega_field} \sim \theta^2$. Note that I advise to round down $\texttt{Omega_field}$ rather than up as it is better to have a cube of intensities full of halos rather than to have the cube's edges missing halos because the angular size is slightly larger than the one from the lightcone. Note also that any value below $\theta^2$ can also be used, as it will just consider a smaller portion of the lightcone.

Observational instrument-related parameters:

9) $\texttt{tobs}$: astropy quantity, observing time [for observational forecast]



Note that if those parameters are not specified, the default ones in $\texttt{params.py}$ will be used. It's therefore important to make sure that all the parameters used for the simulation are the adequate and desired ones. The $\texttt{lim}$ object has a convenient function, $\texttt{update}$, which allows to update its parameters. I typically use this function to set all the important parameters:

In [8]:
m_cii.update(model_par = {'zdex': 0.4,
 'M0': 1900000000.0,
 'Mmin': 20000000000,
 'alpha_MH1': 0.74,
 'alpha_LCII': 0.024,
 'alpha0': -1.412,
 'gamma0': 0.31,
 'BehrooziFile': 'sfr_reinterp.dat'},
            dnu = 2.8*u.GHz,
            nuObs = 270*u.GHz,
            Delta_nu = 40*u.GHz,
            tobs = 40000*u.h,
            Omega_field = 4*u.deg**2,
            catalogue_file = '/home/dongwooc/scratchspace/pprun_hiz_npz/COMAP_z5.8-7.9_960Mpc_seed_13819.npz')